In [1]:
from utils.shark_api import ApiReader,DataManager
import pandas as pd
from utils.calculate import calculate_mining_effort, calculate_time_to_find_block

In [2]:
data_manager = DataManager('../conf')
data_manager.update_data()
# data_manager.start_update_loop(update_interval=60)  # Update every 5 minutes
sharkapi = ApiReader(data_manager)

2024-10-09 13:07:30,688 - utils.shark_api - INFO - Initializing DataManager with config path: ../conf
2024-10-09 13:07:30,798 - utils.shark_api - INFO - DataManager initialized successfully
2024-10-09 13:07:30,798 - utils.shark_api - INFO - --------- UPDATING CORE DATA ---------
2024-10-09 13:07:38,234 - utils.shark_api - INFO - (1/5) Gathered Total Hash Stats
2024-10-09 13:07:43,361 - utils.shark_api - INFO - Total payments: 3114.8200710099823
2024-10-09 13:07:43,362 - utils.shark_api - INFO - (2/5) Gathered Payments Stats
2024-10-09 13:08:13,936 - utils.shark_api - INFO - (3/5) Gathered Pool Stats
2024-10-09 13:08:17,525 - utils.shark_api - INFO - (4/5) Gathered Block Stats
2024-10-09 13:08:23,893 - utils.shark_api - INFO - (5/5) Gathered Live Miner Stats
2024-10-09 13:08:23,894 - utils.shark_api - INFO - --------- UPDATING  COMPLETE --------- 

2024-10-09 13:08:23,894 - utils.shark_api - INFO - ApiReader initialized


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import numpy as np

def prepare_data(data):
    df = pd.DataFrame(data)
    df['created'] = pd.to_datetime(df['created'])
    df = df.sort_values('created')
    df['normalized_shares'] = df['difficulty'].astype(float) * (2**32)
    return df

def calculate_pplns_participation(df, window_size=1000):
    # Ensure index is datetime
    df = df.set_index('created')
    
    # Calculate rolling sum of shares for each miner
    miner_shares = df.groupby('miner')['normalized_shares'].rolling(window=window_size).sum().reset_index()
    
    # Pivot the data to have miners as columns
    miner_shares_pivot = miner_shares.pivot(index='created', columns='miner', values='normalized_shares')
    
    # Fill NaN values with 0
    miner_shares_pivot = miner_shares_pivot.fillna(0)
    
    # Calculate total shares in each window
    total_shares = miner_shares_pivot.sum(axis=1)
    
    # Calculate participation ratios
    participation = miner_shares_pivot.div(total_shares, axis=0)
    
    return participation

def plot_participation_over_time(participation):
    plt.figure(figsize=(12, 6))
    for miner in participation.columns:
        plt.plot(participation.index, participation[miner], label=miner)
    plt.title('Miner Participation Over Time')
    plt.xlabel('Time')
    plt.ylabel('Participation Ratio')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.ylim(0, 1)  # Set y-axis limit from 0 to 1
    plt.tight_layout()
    plt.show()

    # Add a stacked area chart
    plt.figure(figsize=(12, 6))
    participation.plot.area(stacked=True)
    plt.title('Stacked Miner Participation Over Time')
    plt.xlabel('Time')
    plt.ylabel('Participation Ratio')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.ylim(0, 1)  # Set y-axis limit from 0 to 1
    plt.tight_layout()
    plt.show()

    # Print sum of participations
    participation_sum = participation.sum(axis=1)
    print("Summary of participation sums:")
    print(participation_sum.describe())
    
    if not np.allclose(participation_sum, 1, rtol=1e-5, atol=1e-8):
        print("Warning: Not all participation sums are exactly 1. This might be due to floating-point precision.")



def analyze_pplns(data, window_size=1000, block_reward=10):
    df = prepare_data(data)
    participation = calculate_pplns_participation(df, window_size)

    print("Analyzing PPLNS participation...")
    print(f"Total miners: {participation.columns.nunique()}")
    print(f"Date range: {participation.index.min()} to {participation.index.max()}")
    print(f"Total shares: {df['normalized_shares'].sum():,.0f}")
    
    plot_participation_over_time(participation)
    plot_participation_heatmap(participation)
    # plot_participation_distribution(participation)
    # plot_participation_change_rate(participation)
    # plot_cumulative_rewards(participation, block_reward)

    # Calculate some statistics
    avg_participation = participation.mean()
    max_participation = participation.max()
    min_participation = participation.min()

    print("\nParticipation Statistics:")
    print(pd.DataFrame({
        'Average': avg_participation,
        'Max': max_participation,
        'Min': min_participation
    }))

    # Identify top miners
    top_miners = avg_participation.nlargest(5)
    print("\nTop 5 Miners by Average Participation:")
    print(top_miners)

    # Calculate rewards
    total_blocks = len(participation)
    total_rewards = total_blocks * block_reward
    miner_rewards = (participation * block_reward).sum()

    print(f"\nTotal blocks: {total_blocks}")
    print(f"Total rewards: {total_rewards:,.2f}")
    print("\nMiner Rewards:")
    print(miner_rewards.sort_values(ascending=False))



In [6]:
data = sharkapi.get_shares()
data

[{'poolid': 'ErgoSigmanauts',
  'blockheight': 1369945,
  'difficulty': 0.0093039314212643,
  'networkdifficulty': 238003.41379235702,
  'miner': '9gNEKbpPSNfgWkhxsMF4Z9L7uNiXFfsEX4XTVF5XK4kwpQBRsEQ',
  'worker': 'Farn_50_SAFRON',
  'useragent': 'Rigel/1.18.1',
  'ipaddress': '::ffff:94.41.84.85',
  'source': 'ErgoSigmanauts',
  'created': '2024-10-09T17:00:54.315138+00:00'},
 {'poolid': 'ErgoSigmanauts',
  'blockheight': 1369945,
  'difficulty': 0.007818519011041882,
  'networkdifficulty': 238003.41379235702,
  'miner': '9fLytFFzTYALknc2AZ2dRKeg8sLZwe4LX5qAB3FwDysMEeRTHkV',
  'worker': 'rig2',
  'useragent': 'Rigel/1.19.0',
  'ipaddress': '::ffff:129.222.20.52',
  'source': 'ErgoSigmanauts',
  'created': '2024-10-09T17:00:54.352453+00:00'},
 {'poolid': 'ErgoSigmanauts',
  'blockheight': 1369945,
  'difficulty': 0.00363794921875,
  'networkdifficulty': 238003.41379235702,
  'miner': '9iGCyowxwgf6cQef1RfEw6UCYoHJi6q1hZvKvcWyFY1kuLNVE7S',
  'worker': 'FubinbouErgoTrexMiner',
  'useragent

In [8]:
df = prepare_data(data)

In [10]:
df.head()

,poolid,blockheight,difficulty,networkdifficulty,miner,worker,useragent,ipaddress,source,created,normalized_shares
100222,ErgoSigmanauts,1369558,0.014766,214155.741438,9gNEKbpPSNfgWkhxsMF4Z9L7uNiXFfsEX4XTVF5XK4kwpQ...,Farm_43_SAFRON,Rigel/1.18.2,::ffff:94.41.84.85,ErgoSigmanauts,2024-10-09 04:44:20.438517+00:00,6.341929e+07
100223,ErgoSigmanauts,1369558,0.009246,214155.741438,9fy4KkHt9Xavq9R7Wq16euW4JjH1mYwBtV7SNNtAtwnQ6q...,Poltergeist,Rigel/1.19.1,::ffff:208.54.176.157,ErgoSigmanauts,2024-10-09 04:44:20.909919+00:00,3.971240e+07
100224,ErgoSigmanauts,1369558,0.005827,214155.741438,9h6bjuzth7XDNeBxwFjpYSfmUkfBiepE8MGJcvmsVkeCtG...,Poseidon,Rigel/1.19.1,::ffff:72.110.83.232,ErgoSigmanauts,2024-10-09 04:44:21.094932+00:00,2.502481e+07
100225,ErgoSigmanauts,1369558,0.006548,214155.741438,9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2Tu...,Rig_03,Rigel/1.19.1,::ffff:89.187.185.165,ErgoSigmanauts,2024-10-09 04:44:21.589380+00:00,2.812236e+07
100259,ErgoSigmanauts,1369558,0.013845,214155.741438,9gNEKbpPSNfgWkhxsMF4Z9L7uNiXFfsEX4XTVF5XK4kwpQ...,Farm_5,Rigel/1.18.1,::ffff:94.41.84.242,ErgoSigmanauts,2024-10-09 04:44:22.105492+00:00,5.946396e+07


In [11]:
pplns_data = calculate_pplns_participation(df, 1000)

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
analyze_pplns(data, window_size=1000, block_reward=27)

In [ ]:
pplns_data.last